In [1]:
import netCDF4 as nc
import numpy as np
import os
import rasterio
import xarray as xr

## 用rasterio读取tiff文件 边界、中心点

In [ ]:
inpath=r'D:/Gosif_Data/2021/GOSIF_2021001.tif'

with rasterio.open(inpath) as ds:
    sif = ds.read(1)

#边界肯定是没有任何问题的
def tif_bounds():    
    x1, y1 = (ds.bounds.left, ds.bounds.top)
    row, col = ds.index(x1, y1)  # 对应的行列号
    print(f'(投影坐标{x1}, {y1})对应的行列号是({row}, {col})')
    # 根据行列号得到地理坐标
    x1, y1 = ds.xy(row, col)  # 中心点的坐标
    print(f'行列号({row}, {col})对应的中心投影坐标是({x1}, {y1})')
    
    print('-------------------------------------------------')
    x2, y2 = (ds.bounds.right, ds.bounds.bottom)
    row2, col2 = ds.index(x2, y2)  # 对应的行列号
    x2, y2 = ds.xy(row2, col2)  # 中心点的坐标
    print(f'行列号({row2}, {col2})对应的中心投影坐标是({x2}, {y2})')

bounds= ds.bounds #[left, bottom, right, top]
lat = np.linspace(89.975, -89.975, 3600)
lon = np.linspace(-180, 179.95, 7200)


## 批量读取+生成新的nc文件

In [2]:
#%%
def sif_xarray(band1):
    sif=xr.DataArray(band1,dims=['y','x'],coords=[lat, lon])
    return sif.loc[75:45, -15:180]

#%%
def SIF_all(yr):
    inpath = ('D:/Gosif_Data/%d/' %yr)
    filelist = os.listdir(inpath)
    global n, lat_nc, lon_nc
    n=len(filelist)
    
    for i in range(n):
        sif_file=(str(inpath)+filelist[i])
        ds = rasterio.open(sif_file)
        sif = ds.read(1)
        sif_CHN=sif_xarray(sif)
        lat_nc= np.array(sif_CHN.y)
        lon_nc= np.array(sif_CHN.x)
        sif_CHN=np.array(sif_CHN)
        sif_CHN=sif_CHN.reshape(1, 600, 3900)
        
        if(i==0):
            sif_all=sif_CHN
        else:
            sif_all=np.vstack((sif_all, sif_CHN))
        ds.close()
    
    return sif_all

#%%生成新的nc文件
def CreatNC(yr, data):
    doys=np.arange(1, 365, 8)
    
    new_NC = nc.Dataset(r"D:/Gosif_TemBoreal/sif_TemBoreal_%d.nc" %yr, 'w', format='NETCDF4')
    
    new_NC.createDimension('doys', 46)
    new_NC.createDimension('lat', 600)
    new_NC.createDimension('lon', 3900)
    
    var=new_NC.createVariable('sif', 'f', ("doys","lat","lon"))
    new_NC.createVariable('lat', 'f', ("lat"))
    new_NC.createVariable('lon', 'f', ("lon"))
    
    new_NC.variables['sif'][:]=data
    new_NC.variables['lat'][:]=lat_nc
    new_NC.variables['lon'][:]=lon_nc
        
    var.sif= ("AsiaTropic range SIF %d" %yr)
    var.lat_range="[75, 45], 600, 精度：0.05, 边界：[74.975, 45.025]"
    var.lon_range="[-15, 180], 3900, 精度：0.05, 边界：[-15, 180.00]"
    var.data="scale factor和缺测值均未处理"
    var.Fillvalues="32767 (water bodies) and 32766 (lands under snow/ice throughout the year)"
    var.veg_nonveg="annual mean <0 & 32767/32766"
    
    #最后记得关闭文件
    new_NC.close()

In [ ]:
if __name__ == '__main__':
    for yr in range(2001,2022):
        sif_all= SIF_all(yr)
        CreatNC(yr, sif_all)
        print("%d is done!" %yr)